# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to read
data = "../WeatherPy\output"

# Read Data File and store into Pandas DataFrames
weather_data = pd.read_csv(data)
weather_data


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mataura,-46.1927,168.8643,50.49,58.0,85.0,8.68,NZ,1.669870e+09
1,1,albany,42.6001,-73.9662,36.18,73.0,57.0,5.01,US,1.669870e+09
2,2,vaini,-21.2000,-175.2000,82.56,83.0,75.0,9.22,TO,1.669870e+09
3,3,jamestown,42.0970,-79.2353,27.72,97.0,100.0,19.57,US,1.669869e+09
4,4,hasaki,35.7333,140.8333,48.29,76.0,100.0,20.22,JP,1.669870e+09
...,...,...,...,...,...,...,...,...,...,...
73,73,kununurra,-15.7667,128.7333,100.38,16.0,97.0,10.36,AU,1.669870e+09
74,74,guanare,9.0418,-69.7421,77.02,73.0,11.0,8.05,VE,1.669870e+09
75,75,mahebourg,-20.4081,57.7000,81.25,69.0,40.0,6.91,MU,1.669870e+09
76,76,ribeira grande,38.5167,-28.7000,60.78,80.0,5.0,5.99,PT,1.669870e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
len(weather_data)
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  78 non-null     int64  
 1   City        78 non-null     object 
 2   Lat         71 non-null     float64
 3   Lng         71 non-null     float64
 4   Max Temp    71 non-null     float64
 5   Humidity    71 non-null     float64
 6   Cloudiness  71 non-null     float64
 7   Wind Speed  71 non-null     float64
 8   Country     70 non-null     object 
 9   Date        71 non-null     float64
dtypes: float64(7), int64(1), object(2)
memory usage: 6.2+ KB


In [4]:
new = weather_data.dropna()

In [5]:
len(new)
new.head(70)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mataura,-46.1927,168.8643,50.49,58.0,85.0,8.68,NZ,1.669870e+09
1,1,albany,42.6001,-73.9662,36.18,73.0,57.0,5.01,US,1.669870e+09
2,2,vaini,-21.2000,-175.2000,82.56,83.0,75.0,9.22,TO,1.669870e+09
3,3,jamestown,42.0970,-79.2353,27.72,97.0,100.0,19.57,US,1.669869e+09
4,4,hasaki,35.7333,140.8333,48.29,76.0,100.0,20.22,JP,1.669870e+09
...,...,...,...,...,...,...,...,...,...,...
73,73,kununurra,-15.7667,128.7333,100.38,16.0,97.0,10.36,AU,1.669870e+09
74,74,guanare,9.0418,-69.7421,77.02,73.0,11.0,8.05,VE,1.669870e+09
75,75,mahebourg,-20.4081,57.7000,81.25,69.0,40.0,6.91,MU,1.669870e+09
76,76,ribeira grande,38.5167,-28.7000,60.78,80.0,5.0,5.99,PT,1.669870e+09


In [6]:
# Configure gmaps with unique API key
gmaps.configure(api_key=g_key)

In [7]:
# Use the Lat and Lng as locations and Humidity as the weight.

subset = new[['Lat', 'Lng']]
coordinates = [tuple(x) for x in subset.to_numpy()]


In [8]:
# Customize the size of the figure
fig_1_layout = {
    'width': '500px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig_1 = gmaps.figure(layout=fig_1_layout)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig_1.add_layer(markers)
fig_1


Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

In [9]:
humidity = new["Humidity"].astype(float)

In [11]:
# Customize the size of the figure
fig_2_layout = {
    'width': '500px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
# Plot Heatmap
fig_2 = gmaps.figure(layout = fig_2_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 max_intensity=30,
                                 point_radius=3)


# Add layer
fig_2.add_layer(heat_layer)

# Display figure
fig_2

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

preferences = new.iloc[new]



### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
